In [1]:
#!/usr/bin/env python
# coding: utf-8

if 'config' not in locals():
    config = 'configs.config_McNemar'

In [2]:
### initialization ###

import importlib
from _mcnemar import *

print('config file is set to {}'.format(config))

imp=importlib.import_module('_imports')
importlib.reload(imp)

util=importlib.import_module('_utils')
importlib.reload(util)

c=importlib.import_module(config)
importlib.reload(c)

get_ipython().run_line_magic('matplotlib', 'inline')

# param_setup(sys.argv[1:], c)
util.param_setup_ipython(globals(), c)

config file is set to configs.config_McNemar


In [3]:
### reading class labels from files ###


dfs=dict()
dfs['labels']=imp.pd.read_csv('{}data/{}_2_data_labels.csv'.format(c.directory['save'], c.project_name))
# class_labels = [str(label) for label in set(dfs['labels']['label'])]
class_labels = list(set(dfs['labels']['label']))
util.pprint(class_labels)

[False, True]


In [4]:
### calculating McNemar for each class and saving to Excel ###


class_labels=list(map(str, class_labels))
class_labels.append('accuracy')
writer = imp.pd.ExcelWriter(c.directory['save'] + c.project_name + '_McNemar.xlsx', engine='xlsxwriter')
# class_name='accuracy'

for class_label in class_labels:
    fnames = list()
    for experiment in c.experiments:
        fnames.append('experiments/{}/true_pred_results/predictions_{}.csv'.format(experiment, class_label))
        
    p_vals = list()
    for f1 in fnames:
        df1 = imp.pd.read_csv(c.directory['save'] + f1)
        y_target = df1['true']
        y_model1 = df1['pred']
        p_vals_inner = list()

        for f2 in fnames:
            df2 = imp.pd.read_csv(c.directory['save'] + f2)
            if(df1['true'].equals(df2['true'])):
    #             df2 = pd.read_csv(directory + f2)
                y_model2 = df2['pred']
                tb = mcnemar_table(y_target=y_target, 
                   y_model1=y_model1, 
                   y_model2=y_model2)
                chi2, p = mcnemar(ary=tb, corrected=True)
                p_vals_inner.append(p)
            else:   
                print('Target vars are not the same for ' + f1 + '\t' + f2)

        p_vals.append(p_vals_inner)

    df = imp.pd.DataFrame(p_vals, columns=c.experiments)
    df.to_excel(writer, sheet_name=class_label, index=False)
writer.save()

In [5]:
### concatenating classification results and saving to Excel ###


writer = imp.pd.ExcelWriter(c.directory['save'] + c.project_name + '_classification_resutls.xlsx', 
                            engine='xlsxwriter')

for class_label in class_labels:
    df_result=imp.pd.DataFrame()
    for experiment in c.experiments:
        fname='experiments/{}/4_optimal_models_results.csv'.format(experiment, class_label)


        df=imp.pd.read_csv(c.directory['save']+fname)
        df_result=imp.pd.concat([df_result, df.loc[df['metric']==class_label]], axis=0, sort=False)

        
    df_result.fillna(value='-', inplace=True)
    cols=['metric']
    cols_rest=list(df_result.columns)
    del cols_rest[cols_rest.index('metric')]
    cols.extend(cols_rest)
    df_result=df_result[cols]
    df_result=df_result.T        
    df_result.columns=c.experiments   
    df_result.to_excel(writer, sheet_name=class_label, index=True)
writer.save()